In [8]:
import pandas as pd

df = pd.read_csv('/media/r2-d2/E0C494CDC494A6F6/doutorado/rec_info/data/ESTUPRO_CONTEXTO.csv', sep = ';', encoding = 'utf-8')



In [9]:
df

,HISTORICO,CONTEXTO
0,Comparece neste plantão os Policiais Milit...,COTIDIANO
1,Comparece a declarante informando que viveu co...,COTIDIANO
2,"Presente o declarante, informando qu...",COTIDIANO
3,Comparece nesta especializada a senhora Tatian...,COTIDIANO
4,Comparecem nesta delegacia os policiais milita...,COTIDIANO
...,...,...
22911,Comparece neste plantão os policiais militare...,UNICO
22912,"Presente a declarante informando que, na tarde...",UNICO
22913,Comparecem os GCMs Luciane e Detz apresentando...,UNICO
22914,Comparece nesta Unidade Policial o policial mi...,UNICO


In [25]:
# baixando as libs para pre processamento

import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import re
from unidecode import unidecode


nltk.download('stopwords')

stopwords = set(stopwords.words('portuguese'))

stemmer = PorterStemmer()
#nltk.download("wordnet")
#from nltk.stem import WordNetLemmatizer
#lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to /home/r2-d2/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [28]:
# pre processamento

    # Remover caracteres especiais e números
    #text = re.sub('[^A-Za-z\s\']+', '', text)

def preprocess_text(text):

    padrao_arroba = re.compile(r'@\S+')
      # Remover as strings com "@" no início da string
    text = re.sub(padrao_arroba, '', text)

        #remover sites
    #pattern_url = re.compile(r'https?:\/\/.*?[\s+]')

    text = re.sub(r'http\S+', '', text)
    

    # Converter tudo para letras minúsculas
    text = text.lower()

    
    # Remover as tags HTML
    text = re.sub('<[^<]+?>', '', text)    

    text = unidecode(text)
    
    
    # Tokenizar o texto em palavras
    words = nltk.word_tokenize(text)

    # remover palavras que so possua caractere especial
    words = [word for word in words if not re.match('^[^A-Za-z0-9]+$', word)]

    # remover palavras que tenha tamanho 1
    words = [word for word in words if len(word) > 2]

    
    # Remover as stopwords
    words = [word for word in words if word not in stopwords]
    
    # Aplicar o stemming
    words = [stemmer.stem(word) for word in words]
    #words = [lemmatizer.lemmatize(word) for word in words]

    # Juntar as palavras em uma única string
    text = ' '.join(words)
  
    return text

In [29]:
# rodar o pre processamento

def preprocess_collection(docs):

    preprocessed_documents = []

    #for document in docs:
    for index, row in docs.iterrows():
        tema = row['CONTEXTO']
        #nome_arquivo = document['nome_arquivo']
        conteudo = row['HISTORICO']
        
        preprocessed_conteudo = preprocess_text(conteudo)
        
        preprocessed_document = {
            'tema': tema,
        #    'nome_arquivo': nome_arquivo,
            'conteudo': preprocessed_conteudo
        }
        
        preprocessed_documents.append(preprocessed_document)
        
    return preprocessed_documents

In [30]:
df.head(4)

,HISTORICO,CONTEXTO
0,Comparece neste plantão os Policiais Milit...,COTIDIANO
2,"Presente o declarante, informando qu...",COTIDIANO
3,Comparece nesta especializada a senhora Tatian...,COTIDIANO
4,Comparecem nesta delegacia os policiais milita...,COTIDIANO


In [31]:
df = df.sort_values('CONTEXTO')

In [32]:
preprocessed_documents = preprocess_collection(df)

In [33]:
preprocessed_documents

[{'tema': 'COTIDIANO',
  'conteudo': 'comparec nest plantao policiai militar ricardo wilson conduzindo autor marco danilo bezerra segundo relato vitima tayan bezerra santo ano idad fornecido tia anel carla santo jesu levado vitima ate quarto apo pedir deitass cama abrac passou acariciar seio vagina testemunha anel inform depoimento noit hoje volta 21h00 encontrava-s residencia chegou vitima pranto abalada emocionalment repetindo palavra pai depoent levou vitima interior residencia acalm seguida entrevist vitima pergunt havia acontecido pouco ant tayan relat data hoje pouco tempo ant casa genitor marco danilo bezerra pediu deitass cama lado tayan obedeceu seguida marco passou acariciar seio vagina forma maliciosa tayan imediatament tentou desvencilhar braco genitor porem agarrada forca seguida vitima conseguiu libertar correu exterior residencia deixou local tayan diss genitor correu ate portao intuito alcanca-la nao obtev exito tayan diss marco proferiu ameaca contra dizendo contass fa

In [34]:
frases = [o['conteudo'] for o in preprocessed_documents]

In [35]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [36]:
# vetorizando

#temas = [doc["tema"] for doc in preprocessed_documents]
temas = []
for doc in preprocessed_documents:
    temas.append(doc['tema'])


conteudos = [doc["conteudo"] for doc in preprocessed_documents]

vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(conteudos)

In [37]:
preprocessed_documents[10]

{'tema': 'COTIDIANO',
 'conteudo': 'faz-s present nesta delegacia policia especializada ddm genitor menor laura ano julia ano ora representant informando casado mae filha ainda estao separado corpo aproximadament ano cinco mese apo separacao assumiu novo relacionamento alan jose pire ora autor. segundo representant filha estao passando festa final ano companhia hoje sobrinha mesma faixa idad filha dito laura julia havia comentado haviam sido abusada sexualment autor. diant disso representant informa posteriorment convers filha laura dito alan colocado mao dentro calcinha passado mao genitalia momento assistiam film julia dito alan introduzido dedo genitalia alem ter obrigado tocar genitalia masculina menor disseram ainda fato deram varia veze dia diferent nao sabendo precisar quanto tempo vem ocorrendo. requisitou-s exam sexologico cautela menor sendo representant compromet acompanha-la ate instituto perola byngton programa bem quer realizacao exam corpo delito. representant menor sai 

In [39]:
    input_tfidf = vectorizer.transform([preprocessed_documents[10]['conteudo']])
    similaridades = cosine_similarity(input_tfidf, tfidf_matrix)[0]

    resultados = list(zip(similaridades, temas, conteudos))
    resultados.sort(reverse=True)

In [23]:
len(temas)

22916

In [24]:
len(conteudos)

22916

In [3]:
#dataset de teste

import pandas as pd

df_test = pd.read_csv('data/Corona_NLP_test.csv', encoding = "ISO-8859-1")
df_test_new = df_test[['OriginalTweet', 'Sentiment']]

In [4]:
df_test_new

,OriginalTweet,Sentiment
0,TRENDING: New Yorkers encounter empty supermar...,Extremely Negative
1,When I couldn't find hand sanitizer at Fred Me...,Positive
2,Find out how you can protect yourself and love...,Extremely Positive
3,#Panic buying hits #NewYork City as anxious sh...,Negative
4,#toiletpaper #dunnypaper #coronavirus #coronav...,Neutral
...,...,...
3793,Meanwhile In A Supermarket in Israel -- People...,Positive
3794,Did you panic buy a lot of non-perishable item...,Negative
3795,Asst Prof of Economics @cconces was on @NBCPhi...,Neutral
3796,Gov need to do somethings instead of biar je r...,Extremely Negative


In [65]:
df_test_sorted = df_test_new.sort_values('Sentiment')

In [66]:
doc_consultas_preproc = preprocess_collection(df_test_sorted)

In [32]:
doc_consultas_preproc[600]

{'tema': 'Extremely Positive',
 'conteudo': 'grocery store worker also front lines- thankful need protect coronavirus protecting employee serving absolutely necessary role society'}

In [46]:
array_temas = ['COTIDIANO',
'EVENTUAL',
'UNICO']
result_final = {}

for d in array_temas:
    result_final[d] = {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0}

print(result_final)    

{'COTIDIANO': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0}, 'EVENTUAL': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0}, 'UNICO': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0}}


In [47]:
def calcula_map(resultados, tema_verificado):
    acertos = 0
    maps = []
    #for i, doc in enumerate(ranking):
    for i, (similaridade, tema, conteudo) in enumerate(resultados, start=1):
        if tema_verificado == tema:
            acertos +=1
            maps.append(acertos/(i+1))
    return mean(maps)

In [49]:
from statistics import mean

#input_documento = doc_consultas_preproc[150]['conteudo']

p10 = []
p20 = []
p50 = []
p100 = []
map_f = []
tema_atual = ""
for c, input_documento in enumerate(preprocessed_documents):
    #print(c)
    if c == 0:
        tema_atual = input_documento['tema']
    #print(tema_atual)
    acertos = 0     

    input_tfidf = vectorizer.transform([input_documento['conteudo']])
    similaridades = cosine_similarity(input_tfidf, tfidf_matrix)[0]

    resultados = list(zip(similaridades, temas, conteudos))
    resultados.sort(reverse=True)
    #print(len(resultados))
    map_f.append(calcula_map(resultados, input_documento['tema']))
    for rank, (similaridade, tema, conteudo) in enumerate(resultados, start=1):
        #print(rank)
        if input_documento['tema'] == tema:
            #print("acertou")
            acertos += 1
            
        if rank == 10:
            #print("deu 10")
            p10.append(acertos/10.0)
            #print(p10)
        if rank == 20:
            p20.append(acertos/20.0)
        if rank == 50:
            p50.append(acertos/50.0)
        if rank == 100:
            p100.append(acertos/100.0)
        
    
    if tema_atual != input_documento['tema'] or c == len(preprocessed_documents) - 1:
        print("trocou")
#        print(p10)
        result_final[tema_atual]['p10'] = mean(p10)
        result_final[tema_atual]['p20'] = mean(p20)
        result_final[tema_atual]['p50'] = mean(p50)
        result_final[tema_atual]['p100'] = mean(p100)     
        result_final[tema_atual]['map'] = mean(map_f)
        p10 = []
        p20 = []
        p50 = []
        p100 = []
        map_f = []           

    tema_atual = input_documento['tema']        


trocou
trocou
trocou


In [50]:
result_final

{'COTIDIANO': {'p10': 0.6884827535485769,
  'p20': 0.6668456277119953,
  'p50': 0.6547120688387145,
  'p100': 0.6514664999632271,
  'map': 0.6214362304571664},
 'EVENTUAL': {'p10': 0.46650153338051426,
  'p20': 0.42906935598018403,
  'p50': 0.40529370134465675,
  'p100': 0.39811630101439016,
  'map': 0.373279971982555},
 'UNICO': {'p10': 0.2272294887039239,
  'p20': 0.17889417360285376,
  'p50': 0.14316290130796672,
  'p100': 0.12349583828775268,
  'map': 0.0628707304500309}}

In [53]:
import json
with open('/media/r2-d2/E0C494CDC494A6F6/doutorado/rec_info/results/est/est_tfidf.json', 'w') as f:
    json.dump(result_final, f)

In [54]:
import pandas as pd

df = pd.DataFrame(result_final)

In [55]:
df =df.transpose()

In [56]:
df

,p10,p20,p50,p100,map
COTIDIANO,0.688483,0.666846,0.654712,0.651466,0.621436
EVENTUAL,0.466502,0.429069,0.405294,0.398116,0.373280
UNICO,0.227229,0.178894,0.143163,0.123496,0.062871


In [57]:
df.describe()

,p10,p20,p50,p100,map
count,3.000000,3.000000,3.000000,3.000000,3.000000
mean,0.460738,0.424936,0.401056,0.391026,0.352529
std,0.230681,0.244002,0.255801,0.264057,0.279860
min,0.227229,0.178894,0.143163,0.123496,0.062871
25%,0.346866,0.303982,0.274228,0.260806,0.218075
50%,0.466502,0.429069,0.405294,0.398116,0.373280
75%,0.577492,0.547957,0.530003,0.524791,0.497358
max,0.688483,0.666846,0.654712,0.651466,0.621436
